<a href="https://colab.research.google.com/github/SrThunder/Deberes/blob/master/Sistema_de_Predicci%C3%B3n_de_Demanda_con_Prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import streamlit as st
import io
import os
from dotenv import load_dotenv
import openai

# Cargar variables de entorno desde .env
load_dotenv()
# Configurar la clave de la API de OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")

# --- Funciones Principales ---

# 1. Generar Pronósticos de Demanda por Campaña (pronostico_demanda.py)
def generar_pronostico_demanda(datos_historicos, detalles_campana, n_pasos=30):
    """
    Genera pronósticos de demanda para una campaña específica, incluyendo intervalos de confianza,
    utilizando el modelo Prophet para series de tiempo multivariadas.

    Args:
        datos_historicos (pd.DataFrame): DataFrame con datos históricos de ventas, incluyendo columnas como 'fecha', 'ventas', 'campana' y otras variables relevantes.
        detalles_campana (dict): Diccionario con detalles de la campaña actual, incluyendo 'nombre' y 'fecha_inicio'.
        n_pasos (int, opcional): Número de pasos a predecir en el futuro. Por defecto, 30.

    Returns:
        tuple: Un tuple que contiene:
            - pd.DataFrame: DataFrame con las predicciones de demanda, incluyendo las columnas 'fecha', 'prediccion' y los intervalos de confianza de Prophet.
            - str: Una conclusión accionable basada en el pronóstico.
    """
    # Filtrar datos para la campaña específica
    datos_campana = datos_historicos[datos_historicos['campana'] == detalles_campana['nombre']].copy()

    # Asegurarse de que la columna 'fecha' sea de tipo datetime y renombrarla a 'ds'
    datos_campana['ds'] = pd.to_datetime(datos_campana['fecha'])

    # Verificar si hay datos para la campaña
    if datos_campana.empty:
        return pd.DataFrame(columns=['ds', 'yhat', 'yhat_lower', 'yhat_upper']), "No hay datos disponibles para la campaña especificada."

    # Seleccionar las columnas relevantes para Prophet
    columnas_prophet = ['ds', 'ventas'] + [col for col in datos_campana.columns if col not in ['fecha', 'ventas', 'campana']]
    datos_prophet = datos_campana[columnas_prophet].rename(columns={'ventas': 'y'})

    # Crear y entrenar el modelo Prophet
    modelo = Prophet()
    for col in datos_prophet.columns:
        if col not in ['ds', 'y']:
            modelo.add_regressor(col)  # Agregar regresores externos
    modelo.fit(datos_prophet)

    # Crear un DataFrame para las predicciones futuras
    futuro = modelo.make_future_dataframe(periods=n_pasos)
    for col in datos_prophet.columns:
        if col not in ['ds', 'y']:
            futuro[col] = datos_prophet[col].iloc[-n_pasos:].values  # Extender regresores al futuro

    # Generar predicciones
    pronostico = modelo.predict(futuro)

    # Generar conclusión accionable
    aumento_porcentaje = (pronostico['yhat'].iloc[-1] / datos_prophet['y'].mean() - 1) * 100
    if aumento_porcentaje > 10:
        conclusion = f"Se espera un aumento significativo en la demanda. Aumente el inventario en un {aumento_porcentaje:.0f}% para productos clave en la campaña actual."
    elif aumento_porcentaje < -10:
        conclusion = f"Se espera una disminución en la demanda. Considere reducir el inventario en un {abs(aumento_porcentaje):.0f}% para optimizar costos."
    else:
        conclusion = "Se espera una demanda similar a los niveles históricos. Mantenga el inventario actual."

    # Renombrar la columna ds a fecha para mantener la consistencia
    pronostico = pronostico.rename(columns={'ds': 'fecha'})

    return pronostico, conclusion



# 2. Análisis Geográfico de Alto Potencial (analisis_geografico.py)
def analizar_geografia_demanda(datos_historicos):
    """
    Analiza datos geográficos y de ventas para identificar regiones de alta demanda.
    Esta función no cambia significativamente con el cambio a Prophet, pero se mantiene
    para la consistencia de la estructura del sistema.

    Args:
        datos_historicos (pd.DataFrame): DataFrame con datos históricos de ventas, incluyendo columnas como 'region', 'ventas'.

    Returns:
        tuple: Un tuple que contiene:
            - pd.DataFrame: DataFrame con el análisis de demanda por región, incluyendo las columnas 'region' y 'demanda_proyectada'.
            - str: Una recomendación accionable basada en el análisis geográfico.
    """
    # Verificar si la columna 'region' existe
    if 'region' not in datos_historicos.columns:
        raise ValueError("La columna 'region' no existe en el DataFrame.")

    # Calcular la demanda total por región
    demanda_por_region = datos_historicos.groupby('region')['ventas'].sum().reset_index()

    # Proyectar la demanda (ejemplo simple: crecimiento promedio)
    crecimiento_promedio = datos_historicos['ventas'].pct_change().mean()
    demanda_por_region['demanda_proyectada'] = demanda_por_region['ventas'] * (1 + crecimiento_promedio)

    # Identificar la región de mayor crecimiento
    region_max_crecimiento = demanda_por_region.loc[demanda_por_region['demanda_proyectada'].idxmax()]
    crecimiento_porcentaje = (region_max_crecimiento['demanda_proyectada'] / demanda_por_region['ventas'].mean() - 1) * 100
    recomendacion = f"Dirija campañas hacia la región {region_max_crecimiento['region']}, donde se espera un crecimiento del {crecimiento_porcentaje:.0f}%."

    return demanda_por_region, recomendacion



# 3. Identificación de Productos Estacionales (estacionalidad.py)
def identificar_productos_estacionales(datos_historicos):
    """
    Identifica productos con demanda estacional y sus ciclos.
    Esta función no cambia con el cambio a Prophet, pero se mantiene para la consistencia.

    Args:
        datos_historicos (pd.DataFrame): DataFrame con datos históricos de ventas, incluyendo columnas como 'fecha', 'producto', 'ventas'.

    Returns:
        tuple: Un tuple que contiene:
            - dict: Un diccionario donde las claves son los productos y los valores son DataFrames con el análisis estacional ('mes' y 'ventas_promedio').
            - dict: Un diccionario de recomendaciones accionables por producto.
    """
    # Asegurarse de que la columna 'fecha' sea de tipo datetime
    datos_historicos['fecha'] = pd.to_datetime(datos_historicos['fecha'])

    # Verificar si las columnas 'producto' y 'ventas' existen
    if 'producto' not in datos_historicos.columns or 'ventas' not in datos_historicos.columns:
        raise ValueError("El DataFrame debe contener las columnas 'producto' y 'ventas'.")

    # Agrupar datos por producto y mes
    datos_historicos['mes'] = datos_historicos['fecha'].dt.month
    ventas_mensuales = datos_historicos.groupby(['producto', 'mes'])['ventas'].mean().reset_index()

    # Calcular ventas promedio por mes para cada producto
    productos_estacionales = {}
    recomendaciones = {}
    for producto in ventas_mensuales['producto'].unique():
        datos_producto = ventas_mensuales[ventas_mensuales['producto'] == producto].copy()
        datos_producto['mes_nombre'] = datos_producto['mes'].apply(lambda x: calendar.month_abbr[x]) # Convertir número de mes a nombre
        productos_estacionales[producto] = datos_producto
        # Identificar el mes de mayor venta
        mes_max_venta = datos_producto.loc[datos_producto['ventas'].idxmax()]
        recomendaciones[producto] = f"Se espera que el producto {producto} tenga su mayor demanda en {mes_max_venta['mes_nombre']}."

    return productos_estacionales, recomendaciones



# 4. Impacto de Promociones (impacto_promociones.py)
def analizar_impacto_promociones(datos_historicos, datos_promociones):
    """
    Analiza el impacto de las promociones en la demanda.
    Esta función no cambia significativamente con el cambio a Prophet.

    Args:
        datos_historicos (pd.DataFrame): DataFrame con datos históricos de ventas, incluyendo columnas como 'fecha', 'ventas', y opcionalmente 'promocion_id'.
        datos_promociones (pd.DataFrame): DataFrame con datos de promociones, incluyendo columnas como 'promocion_id', 'fecha_inicio', 'fecha_fin', 'tipo'.

    Returns:
        tuple: Un tuple que contiene:
            - pd.DataFrame: DataFrame con el análisis del impacto de las promociones.
            - dict: Un diccionario con conclusiones accionables por promoción.
    """
    # Asegurarse de que las columnas 'fecha' sean de tipo datetime
    datos_historicos['fecha'] = pd.to_datetime(datos_historicos['fecha'])
    if not datos_promociones.empty:
        datos_promociones['fecha_inicio'] = pd.to_datetime(datos_promociones['fecha_inicio'])
        datos_promociones['fecha_fin'] = pd.to_datetime(datos_promociones['fecha_fin'])

    # Si no hay promociones, retornar dataframes vacíos
    if datos_promociones.empty:
        return pd.DataFrame(), {}

    # Fusionar datos de ventas y promociones (asumiendo que hay una columna 'promocion_id' en datos_historicos)
    datos_combinados = pd.merge(datos_historicos, datos_promociones, on='promocion_id', how='left')

    # Calcular el impacto de cada promoción
    impacto_promociones = datos_combinados.groupby('promocion_id')['ventas'].mean().reset_index()
    impacto_promociones = impacto_promociones.rename(columns={'ventas': 'ventas_promedio'})

    # Calcular ventas promedio sin promoción (para comparar)
    ventas_sin_promocion = datos_historicos[datos_historicos['promocion_id'].isnull()]['ventas'].mean()
    impacto_promociones['ventas_sin_promocion'] = ventas_sin_promocion

    # Calcular el incremento porcentual en ventas
    impacto_promociones['incremento_porcentaje'] = ((impacto_promociones['ventas_promedio'] / ventas_sin_promocion) - 1) * 100

    # Generar conclusiones accionables
    conclusiones = {}
    for _, row in impacto_promociones.iterrows():
        promocion_id = row['promocion_id']
        incremento = row['incremento_porcentaje']
        if incremento > 0:
            conclusiones[promocion_id] = f"Se estima un incremento del {incremento:.0f}% en las ventas debido a la promoción {promocion_id}."
        else:
            conclusiones[promocion_id] = f"La promoción {promocion_id} no tuvo un impacto positivo en las ventas."

    return impacto_promociones, conclusiones



# 5. Carga y Validación de Datos (carga_datos.py)
def cargar_y_validar_datos(archivo):
    """
    Carga y valida datos desde un archivo CSV o Excel.

    Args:
        archivo (str o BytesIO): Ruta al archivo de datos o BytesIO object.

    Returns:
        pd.DataFrame: DataFrame con los datos cargados, o None si hay un error.
        str: Mensaje indicando el resultado de la carga y validación.
    """
    try:
        if isinstance(archivo, str):  # Si es una ruta de archivo
            if archivo.endswith('.csv'):
                datos = pd.read_csv(archivo)
            elif archivo.endswith(('.xls', '.xlsx')):
                datos = pd.read_excel(archivo)
            else:
                return None, "Formato de archivo no soportado. Por favor, use CSV o Excel."
        elif isinstance(archivo, io.BytesIO): #Si es un objeto BytesIO
             # Determinar el formato basado en el nombre del archivo (si está disponible)
            # Esto es una simplificación, st.file_uploader no siempre proporciona el nombre real
            if hasattr(archivo, 'name'):
                if archivo.name.endswith('.csv'):
                    datos = pd.read_csv(archivo)
                elif archivo.name.endswith(('.xls', '.xlsx')):
                    datos = pd.read_excel(archivo)
                else:
                    return None, "Formato de archivo no soportado. Por favor, use CSV o Excel."
            else:
                return None, "No se pudo determinar el formato del archivo.  Por favor, use CSV o Excel."

        else:
            return None, "Tipo de archivo no soportado. Por favor, use CSV o Excel."

        # Validación básica de columnas (puedes agregar validaciones más específicas)
        if not all(col in datos.columns for col in ['fecha', 'ventas']):
            return None, "El archivo debe contener las columnas 'fecha' y 'ventas'."

        # Convertir la columna 'fecha' a datetime, si existe
        if 'fecha' in datos.columns:
            try:
                datos['fecha'] = pd.to_datetime(datos['fecha'])
            except ValueError:
                return None, "La columna 'fecha' no tiene un formato de fecha válido."

        return datos, "Datos cargados y validados exitosamente."
    except Exception as e:
        return None, f"Error al cargar el archivo: {e}"



# 6. Generación de Insights con OpenAI (insights.py)
def generar_insights_openai(resultados, funcion_nombre):
    """
    Genera insights y recomendaciones usando la API de OpenAI.

    Args:
        resultados (pd.DataFrame o dict): Resultados de una de las funciones de análisis.
        funcion_nombre (str): Nombre de la función que generó los resultados ('generar_pronostico_demanda', 'analizar_geografia_demanda', 'identificar_productos_estacionales', 'analizar_impacto_promociones').

    Returns:
        str: Un texto con los insights y recomendaciones generados por la API de OpenAI.
    """
    if resultados is None or (isinstance(resultados, pd.DataFrame) and resultados.empty) or (isinstance(resultados, dict) and not resultados):
        return "No hay resultados para analizar."

    prompt = f"Analiza los siguientes resultados de la función {funcion_nombre} y proporciona insights y recomendaciones accionables para un Director de Ventas o Marketing:\n\n"

    if funcion_nombre == 'generar_pronostico_demanda':
        prompt += f"Datos: {resultados.to_string()}\n\n"
        prompt += "Interpreta las predicciones de demanda de la serie de tiempo, los intervalos de confianza, y sugiere acciones para optimizar el inventario y la planificación de la campaña.  Considera que se está utilizando el modelo Prophet."
    elif funcion_nombre == 'analizar_geografia_demanda':
        prompt += f"Datos: {resultados.to_string()}\n\n"
        prompt += "Identifica las regiones con mayor potencial de crecimiento y sugiere estrategias para enfocar los esfuerzos de marketing y ventas."
    elif funcion_nombre == 'identificar_productos_estacionales':
        prompt += f"Datos: {resultados}\n\n" # Los resultados de estacionalidad ya están en formato dict
        prompt += "Describe los patrones de estacionalidad de los productos y recomienda acciones para optimizar la promoción y el inventario a lo largo del año."
    elif funcion_nombre == 'analizar_impacto_promociones':
        prompt += f"Datos: {resultados.to_string()}\n\n"
        prompt += "Evalúa la efectividad de las promociones y sugiere mejoras para futuras campañas."
    else:
        return "Función no soportada."

    try:
        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",  # Puedes ajustar el motor según tus necesidades
            prompt=prompt,
            max_tokens=200,  # Limita la longitud de la respuesta
            temperature=0.7,  # Ajusta la creatividad de la respuesta
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return f"Error al contactar a la API de OpenAI: {e}"



# --- Interfaz de Usuario con Streamlit (main.py) ---
def main():
    st.title("Sistema de Predicción de Demanda")

    # Cargar datos
    archivo = st.file_uploader("Cargar Datos Históricos (CSV, Excel)", type=["csv", "xls", "xlsx"])
    if archivo is not None:
        # Para st.file_uploader, el archivo se carga como un objeto BytesIO
        datos_historicos, mensaje = cargar_y_validar_datos(archivo)
        if datos_historicos is not None:
            st.success(mensaje)

            # Agregamos un st.cache_data() para que Streamlit no re-ejecute el análisis cada vez.
            @st.cache_data()
            def calcular_resultados(datos_historicos):
                # Asegurarse de que los datos no son None
                if datos_historicos is None or datos_historicos.empty:
                    return None, None, None, None

                detalles_campana_ejemplo = {'nombre': datos_historicos['campana'].unique()[0], 'fecha_inicio': datos_historicos['fecha'].min()} # Ejemplo
                resultados_pronostico, _ = generar_pronostico_demanda(datos_historicos, detalles_campana_ejemplo, n_pasos=30) #n_pasos
                analisis_region, _ = analizar_geografia_demanda(datos_historicos)
                productos_estacionales, _ = identificar_productos_estacionales(datos_historicos)
                # Ejemplo de datos de promociones (puedes cargar esto también si es necesario)
                datos_promociones_ejemplo = pd.DataFrame({
                    'promocion_id': [1, 2],
                    'fecha_inicio': pd.to_datetime(['2023-01-01', '2023-03-01']),
                    'fecha_fin': pd.to_datetime(['2023-01-31', '2023-03-31']),
                    'tipo': ['Descuento', '2x1']
                })
                impacto_promociones, _ = analizar_impacto_promociones(datos_historicos, datos_promociones_ejemplo)
                return resultados_pronostico, analisis_region, productos_estacionales, impacto_promociones

            resultados_pronostico, analisis_region, productos_estacionales, impacto_promociones = calcular_resultados(datos_historicos)

            # --- Pestañas ---
            tab1, tab2, tab3, tab4 = st.tabs(["Pronósticos", "Análisis Geográfico", "Productos Estacionales", "Impacto Promociones"])

            with tab1:
                st.header("Pronósticos de Demanda por Campaña")
                if resultados_pronostico is not None and not resultados_pronostico.empty:
                    # Crear el gráfico con Streamlit
                    st.subheader(f'Pronóstico de Demanda para la Campaña: {datos_historicos["campana"].unique()[0]}')
                    # Asegurarse de que 'fecha' esté en el DataFrame
                    if 'fecha' in resultados_pronostico:
                        chart_data = resultados_pronostico[['fecha', 'yhat', 'yhat_lower', 'yhat_upper']].set_index('fecha')
                        st.line_chart(chart_data)
                    else:
                        st.warning("La columna 'fecha' no está presente en los resultados del pronóstico.")

                    # Mostrar la conclusión de OpenAI
                    insight_pronostico = generar_insights_openai(resultados_pronostico, 'generar_pronostico_demanda')
                    st.write(insight_pronostico)
                else:
                    st.warning("No hay datos disponibles para generar el pronóstico.")

            with tab2:
                st.header("Análisis Geográfico de Alto Potencial")
                if analisis_region is not None and not analisis_region.empty:
                    # Crear el gráfico de barras con Streamlit
                    st.subheader('Demanda Proyectada por Región')
                    st.bar_chart(analisis_region.set_index('region'))
                    insight_geografia = generar_insights_openai(analisis_region, 'analizar_geografia_demanda')
                    st.write(insight_geografia)
                else:
                    st.warning("No hay datos disponibles para el análisis geográfico.")

            with tab3:
                st.header("Identificación de Productos Estacionales")
                if productos_estacionales:
                    for producto, datos_producto in productos_estacionales.items():
                        st.subheader(f'Ventas Mensuales Promedio para {producto}')
                        st.bar_chart(datos_producto.set_index('mes_nombre'))
                    insight_estacionalidad = generar_insights_openai(productos_estacionales, 'identificar_productos_estacionales')
                    st.write(insight_estacionalidad)
                else:
                    st.warning("No hay datos disponibles para el análisis de estacionalidad.")

            with tab4:
                st.header("Impacto de Promociones")
                if impacto_promociones is not None and not impacto_promociones.empty:
                    # Crear el gráfico de barras con Streamlit
                    st.subheader('Impacto de las Promociones en Ventas')
                    chart_data_promociones = impacto_promociones[['promocion_id', 'ventas_promedio', 'ventas_sin_promocion']].set_index('promocion_id')
                    st.bar_chart(chart_data_promociones)
                    insight_promociones = generar_insights_openai(impacto_promociones, 'analizar_impacto_promociones')
                    st.write(insight_promociones)
                else:
                    st.warning("No hay datos disponibles para el análisis de promociones.")
        else:
            st.error(mensaje)

if __name__ == "__main__":
    main()